In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import re

## Day 19

In [117]:
file19 = open('./input_files/day19_input', 'r') 
input19_lines = file19.readlines() 

In [118]:
def split_rules(rules):
        
    if '"a"' in rules:
        return ['a']
    elif '"b"' in rules:
        return ['b']
    else:
        rules_idx = re.findall(r'\d+', rules)
        return [int(r) for r in rules_idx]
    
def subst_list(main, origin, subst):
    """ Subsitutes every occurace of 'origin' in main with subst. """
    eq_origin = [e == origin for e in main]
    
    new_main = list(main)
    for idx, eq in enumerate(eq_origin):
        if eq:
            new_main = new_main[:idx] + subst + new_main[idx+1:]
    
    return new_main

In [119]:
## Create Rules table
rules_nbr = []
rules_table = []
for line in input19_lines[:131]:
    
    line = line.replace('\n', '')
    nbr, rules = line.split(':')

    rules_nbr += [int(nbr)]
    
    if '|' in rules:
        rules_split = rules.split('|')
    else:
        rules_split = [rules]
    
    rl_list = []
    for rl in rules_split:
        rl_list += [split_rules(rl)]
    
    rules_table += [rl_list]
    
rules_nbr  = np.array(rules_nbr)
sorted_idx = np.argsort(rules_nbr)

rules_table = [rules_table[idx] for idx in sorted_idx]
print('Number of path splits in rule book:', sum([len(rule) > 1 for rule in rules_table]))

# Read messages
messages = []
for line in input19_lines[132:]:
    
    line = line.replace('\n', '')
    messages += [line]
    
MAX_SEQ_LENGTH = max([len(m) for m in messages])
print('Length of longest messagae:',  MAX_SEQ_LENGTH)

Number of path splits in rule book: 115
Length of longest messagae: 88


In [121]:
2**43

8796093022208

In [116]:
rules_table

[[[4, 1, 5]],
 [[2, 3], [3, 2]],
 [[4, 4], [5, 5]],
 [[4, 5], [5, 4]],
 [['a']],
 [['b']]]

### Part A

In [122]:
def next_branch(path_id):

    bin_path_id = list(bin(path_id)[2:])
    bin_path_id = [int(s) for s in bin_path_id]
    
    for direction in reversed(bin_path_id):
        yield direction

    while True:
        yield 0

In [123]:
def walk_tree(init_rule_seq, rule_book, path_generator):
    """ Walk the subsitiution rule tree recursively along a trajectory given by path_nbr """
    
    rule_seq = list(init_rule_seq)
    while (not all([(e == 'a') | (e == 'b') for e in rule_seq])):
    
        new_rule_seq = list(rule_seq)
        for rule in rule_seq:
            if (rule == 'a') | (rule == 'b'):
                continue
            else:                
                subs_rules = rule_book[rule]
                if len(subs_rules) == 1:
                    new_rule_seq = subst_list(new_rule_seq, rule, subs_rules[0])
                elif len(subs_rules) == 2:
                    new_rule_seq = subst_list(new_rule_seq, rule, subs_rules[next(path_generator)])
                else:
                    print('Error: Nbr of subst rules > 2. Len =', len(subs_rule), subs_rule)
                    
        rule_seq = list(new_rule_seq)
        if len(rule_seq) > MAX_SEQ_LENGTH:
            return ''
        
    return ''.join(rule_seq)
    

In [125]:
init_rule_seq = rules_table[0][0]

for path_id in range(10):
    code = walk_tree(init_rule_seq, rules_table, next_branch(path_id))
    print(path_id, '\t', code)

0 	 
1 	 aaaabbabababbabbbababbabaabbabbabababaaaabababbaabab
2 	 aaabaaaaaaaaaaaaaaaaaabaabaaaaaabaaabaaabaaaaaabaaabaaaabaaaaaaaabbabbbababbaabab
3 	 
4 	 
5 	 aaaabbabababbabbbababbabaabbabbaabaaaaabbbabbaa
6 	 aaabaaaaaaaaaaaaaaaaaabaabaaaaaabaaabaaabaaaaaabaaabaaaabaaaaaaaabbabbbbbabbaa
7 	 
8 	 bbbabbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbabbbabbbbbbbbbbabbbbabbbbaaabaaaaaabbaabab
9 	 bbabababaaabbabbaabaaaaabababbaabab


In [128]:
walk_tree(init_rule_seq, rules_table, next_branch(8796093022209))

'aaaabbabababbabbbababbabaabbabbabababaaaabababbaabab'

In [124]:
# Check which of the messages are valid. That is
for message in messages:
    
    while message != valid_message:
        valid_message = walk_tree(init_rule_seq, rules_table, next_branch(path_id))
        #print(path_id, '\t', valid_message)

NameError: name 'valid_message' is not defined

#### Old method: attempt to make all possible strings that fullfil the rules.

In [ ]:
rule_seq_list = list(rules_table[0])
rule_seq_flat = [False]*len(rule_seq_list)
total_ab = sum(rule_seq_flat)
idx = 0

new_rule_seq_list = list([rule_seq_list])
while total_ab != len(rule_seq_flat):

    print(idx, '  -  ', rule_seq_list)   
    
    new_rule_seq_list = []
    for rule_seq_idx, rule_seq in enumerate(rule_seq_list):
        
        # Subsitute all rules that do not contain a split.
        new_rule_seq = list(rule_seq)
        for rule_idx, rule in enumerate(rule_seq):
            if (rule == 'a') | (rule == 'b'):
                continue
            else:                
                subs_rules = rules_table[rule]
                if len(subs_rules) == 1:
                    new_rule_seq = subst_list(new_rule_seq, rule, subs_rules[0])
        
        # Subsitute all rules that split.
        temp_nrs_list = [new_rule_seq]
        for rule_idx, rule in enumerate(rule_seq):
            if (rule == 'a') | (rule == 'b'):
                continue
            else:                
                subs_rules = rules_table[rule]
                if len(subs_rules) == 2:
                    temp = []
                    for nrs in temp_nrs_list:
                        new_rule_seq1 = subst_list(nrs, rule, subs_rules[0])
                        new_rule_seq2 = subst_list(nrs, rule, subs_rules[1])
                        temp += [new_rule_seq1]
                        temp += [new_rule_seq2]
                    
                    temp_nrs_list = list(temp)
                    
        new_rule_seq_list += temp_nrs_list
                    
    #print(new_rule_seq_list)
                        
    rule_seq_list = list(new_rule_seq_list)
    rule_seq_flat = [(l == 'a') | (l == 'b') for ll in rule_seq_list for l in ll]
    total_ab = sum(rule_seq_flat)
    
    idx += 1
    
new_rule_seq_list